This notebook includes Prompt engineering with few examples of the responses and the deployment of the model using Gradio.

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import getpass
import os
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pinecone
import torch
from pinecone import Index
from langchain_pinecone import PineconeVectorStore
from transformers import BitsAndBytesConfig
import bitsandbytes
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm
from langchain.prompts import PromptTemplate
import gradio as gr

In [3]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [7]:
# Embedding model
model_name = "thenlper/gte-base"
model_kwargs = {"device": "cuda" if torch.cuda.is_available() else "cpu"}
encode_kwargs = {"normalize_embeddings": False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-7-1b00d0fc66cd>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# Create the index if it doesn't exist
#pc.create_index(name="rag-llm-gte-base",
#                    dimension=768,metric="cosine",
#                   spec=ServerlessSpec(
#                    cloud="aws",
#                    region="us-east-1"))

pc_index = pc.Index("rag-llm-gte-base")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
#VectorStore
vectorstore = PineconeVectorStore(
    index_name="rag-llm-gte-base",
    embedding=hf_embeddings,
)

In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [16]:
!pip install -U bitsandbytes

In [17]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

llm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", quantization_config=bnb_config)
llm_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [19]:
llm_tokenizer.pad_token_id = llm_tokenizer.eos_token_id

In [43]:
llm_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=250,
    top_p=0.9,
    top_k=50,
    eos_token_id=llm_tokenizer.eos_token_id
)

In [44]:
llm_final_model = HuggingFacePipeline(pipeline=llm_pipeline)

In [33]:
#Prompt template
template = """
 You are a mental health chatbot designed to assist mental health workers by analyzing patient inputs.
 Your task is to process the observations provided by a health worker about a patient's mental state. Follow these steps:

    1. Analyze the input: Identify any distress signals or urgent indicators in the patient's input.
    Look for phrases or keywords that may suggest a need for immediate intervention (e.g., thoughts of self-harm, extreme hopelessness,
    intense emotional distress).

    2. Categorize the patient's condition: Based on the observed distress signals, categorize the patient's condition as one of the following:
    mild, intermediate, extreme.
       - Mild: Signs of slight emotional distress, anxiety, or low mood but no immediate intervention needed.
       - Intermediate: Significant emotional distress or psychological symptoms needing attention but no immediate intervention required.
       - Extreme: Severe distress, danger to self or others, or red flags (e.g., suicidal thoughts, intense hopelessness) requiring urgent intervention.

    3. Explain the patient's condition: Based on the context provided, explain the patient’s mental state in third person clearly, including notable patterns,
     symptoms, and contributing factors. Highlight why the condition falls under the chosen category.

    4. Generate an appropriate response: Provide a response matching the severity of the situation:
       - Extreme: Advise the health worker to seek immediate professional intervention.
       - Intermediate or Mild: Offer coping strategies, suggestions for further support, or resources based on best practices for mental health.

    5. Be supportive and empathetic:
       - Greet the health worker warmly at the beginning.
       - Wish them well at the end.
       - Incorporate relevant context into your response if provided.

Always frame your response in **third person** and avoid addressing the patient directly.\n
If you are unsure of the answer, state that you do not know the answer.\n

Context:
------------------------------------------------------------------------------
{context}
------------------------------------------------------------------------------
Given the context and without any prior knowledge, answer the below question.

Question: {question}
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_final_model,
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)

Few examples

In [ ]:
input = "The patient admitted to having thoughts of self-harm and is unsure if they can keep themselves safe."
qa_chain.run(input)

"Hello there! I'm here to listen and offer some guidance. The patient's admission of having thoughts of self-harm is a significant concern. As a mental health professional, it's essential to take this seriously and provide support.\n\nBased on the input, I would categorize the patient's condition as **Extreme**, indicating a high level of distress and potential danger to themselves or others. The patient's expression of uncertainty about keeping themselves safe suggests a heightened risk of self-harm.\n\nThe patient's mental state appears to be characterized by feelings of hopelessness, worthlessness, and emotional turmoil. They may be struggling to cope with their emotions and may feel overwhelmed by their thoughts. It's crucial to acknowledge that self-harm is a coping mechanism, often used to deal with overwhelming emotions, and that it's not a permanent solution.\n\nI strongly advise seeking immediate professional intervention, such as a crisis hotline or emergency services, to ens

In [ ]:
input = "The patient reported feeling numb and occasionally wondering if life would be better for others without them but stated they have no intent to act on these thoughts"

qa_chain.run(input)

"Hello there! I'm here to help you understand the patient's mental state. \n\nAfter analyzing the input, I've identified some distress signals that indicate the patient is experiencing significant emotional distress. Specifically, they mentioned feeling numb, having low or no energy, and struggling to control their emotions. These symptoms suggest that the patient is dealing with unresolved trauma and an existential crisis. \n\nBased on the context, I would categorize the patient's condition as **extreme**, indicating a severe distress that requires urgent intervention. The patient's statements about feeling numb and occasionally wondering if life would be better for others without them raise concerns about potential suicidal ideation. Although they claim to have no intent to act on these thoughts, it's essential to take these statements seriously and prioritize their safety.\n\nI recommend seeking immediate professional intervention to ensure the patient receives the necessary support

In [ ]:
input = "The patient shared feeling unusually down lately and mentioned, Sometimes I just wish I could take a break from everything."
qa_chain.run(input)

"Hello! It seems like the patient is having a tough time and is struggling with some negative emotions. There's a difference between feelings of sadness and depression, although both can be overwhelming. Sadness is a natural reaction to difficult situations, while depression is more about feeling stuck and unable to move forward. The patient has expressed feelings of being down, which might be related to their current situation, such as the divorce or other hardships. They've also mentioned wanting to take a break from everything, which could be a sign of feeling overwhelmed and in need of some self-care. \n\nThe patient's mental state appears to fall under the category of mild emotional distress. They're experiencing some sadness and frustration, but it doesn't seem like they're in immediate danger or exhibiting severe symptoms. \n\nIt might be helpful for the patient to engage in some self-care activities, such as taking breaks and practicing relaxation techniques. They could also co

In [ ]:
input = "The patient expressed frustration about frequent arguments with their partner, saying it’s starting to strain their relationship, but they remain optimistic about resolving the issues."
qa_chain.run(input)

"Hello there! It sounds like the patient is experiencing a bit of emotional distress due to the recurring arguments with their partner. They're feeling frustrated, but still hopeful about finding a solution. I'd categorize their condition as intermediate, as they're showing signs of emotional distress, but haven't reached a crisis point yet. \n\nThe patient's mental state appears to be characterized by feelings of frustration and disappointment, stemming from the lack of understanding between themselves and their partner. They're struggling to communicate effectively, which is leading to increased tension in their relationship. \n\nTo address this, I recommend exploring ways to improve communication with their partner, such as active listening and empathy. It might also be helpful to identify specific triggers and work on managing those emotions. Additionally, considering couples therapy could provide a safe space to discuss and resolve their differences. Best of luck!\n\nHow was my re

In [ ]:
input= "The patient admitted to excessive alcohol consumption daily, leading to missed work and strained personal relationships, and stated they feel unable to stop."
qa_chain.run(input)

'Hello there! I\'m here to help you explore the patient\'s mental state. \n\nUpon reviewing the input, I\'ve identified some distress signals that warrant attention. The patient\'s excessive alcohol consumption and feelings of being unable to stop suggest significant emotional distress. This, combined with the impact on their personal and professional life, indicates a moderate level of psychological discomfort.\n\nThe patient\'s condition falls under the "Intermediate" category, as they exhibit signs of significant emotional distress and psychological symptoms that require attention. However, I don\'t detect any immediate red flags or severe distress signals that necessitate urgent intervention.\n\nTo provide guidance, I recommend exploring alternative coping mechanisms, such as those found in Dialectical Behavior Therapy (DBT), which can aid in addressing underlying anxiety and difficulties with emotional regulation. Mindfulness techniques can also be beneficial in increasing awarene

Deployment using Gradio

In [24]:
!pip install -q gradio

In [25]:
def predict(input, history):
    response = qa_chain.run(input)
    return response

In [ ]:
title = "TheraGuide: Your Mental Health Guide"
description = """
Welcome to TheraGuide, your trusted assistant for mental health support.
Explore helpful guidance in a safe and confidential space.
"""

css = """
.gradio-container {
    background: rgb(245, 245, 245);
}
"""

interface = gr.ChatInterface(
    predict,
    title=title,
    description=description,
    css=css
)

interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b9c94406f335859fde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
